In [ ]:
import os
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import kurtosis

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.swa_utils import AveragedModel, SWALR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
BASE_DIR = Path(r"D:\DRIVE\UNI\moviZ\TERM8\ANN-article--AzRiz\OurWorks\Data-description\phase1")
REVERSED_FOLDER = "reversed"
CACHE_DIR = Path("cache_article_ready_vfinal")
CACHE_DIR.mkdir(exist_ok=True, parents=True)

TARGET_LEN = 2048
SMOOTH_KERNEL = 7
BATCH_SIZE = 64
ACCUM_STEPS = 2
EPOCHS = 300
LR = 3e-4
MIN_LR = 1e-6
WARMUP_EPOCHS = 10
WEIGHT_DECAY = 1e-4
PATIENCE = 100
VAL_RATIO = 0.15
TEST_RATIO = 0.15
SEED = 42
AUGMENT = True
DEBUG = False
RUL_CLIP = 130
USE_HUBER_LOSS = True
USE_TTA = True
SWA_START = 150

<div dir="rtl">

## Configurations

این بخش متغیرهای ثابت و hyperparameterهای مدل را تعریف می‌کند، شامل مسیرهای directory برای داده‌ها و cache، طول سیگنال هدف (TARGET_LEN)، اندازه batch، تعداد epochs، learning rate، و flagهایی برای فعال‌سازی ویژگی‌هایی مثل data augmentation، Huber loss، و test-time augmentation (TTA).

متغیرها با مقادیر مشخص initialize می‌شوند. Path از pathlib برای مدیریت مسیرهای فایل به‌صورت cross-platform استفاده می‌کند. دستور mkdir دایرکتوری cache را اگر وجود نداشته باشد ایجاد می‌کند.

مسیرهای فایل و hyperparameterها تعریف می‌شوند. برای مثال، CACHE_DIR برای ذخیره dataهای preprocess‌شده تنظیم می‌شود، و BATCH_SIZE=64 برای بهینه‌سازی memory و سرعت train انتخاب می‌شود.

تعریف متغیرها در یک مکان مرکزی، کد را modular و قابل تنظیم می‌کند. مثلاً تغییر LR یا EPOCHS بدون نیاز به ویرایش کل کد ممکن است. مقادیر مثل EPOCHS=300 برای اطمینان از convergence کافی مدل انتخاب شده‌اند، و SEED=42 برای reproducibility نتایج است. flagهایی مثل AUGMENT=True برای افزایش generalization مدل و کاهش overfitting فعال هستند.

</div>

In [ ]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

<div dir="rtl">

## Seed and Device Setup

این بخش seed را برای libraryهای random، numpy و torch تنظیم می‌کند تا نتایج reproducible باشند و دستگاه محاسباتی (GPU یا CPU) را انتخاب می‌کند.

تابع seed روی هر library اعمال می‌شود تا عملیات تصادفی مثل shuffling یا weight initialization یکسان باشند. اگر CUDA در دسترس باشد، تنظیمات اضافی برای determinism اعمال می‌شود. سپس device بر اساس در دسترس بودن GPU انتخاب می‌شود.

ابتدا seedها تنظیم می‌شوند، سپس در دسترس بودن GPU بررسی می‌شود، و device مناسب (cuda یا cpu) allocate می‌شود. پیام چاپ‌شده device انتخاب‌شده را نشان می‌دهد.

تنظیم seed برای اطمینان از reproducibility نتایج ضروری است، چون deep learning شامل عملیات تصادفی است. استفاده از GPU سرعت محاسبات را به شدت افزایش می‌دهد، به‌ویژه برای neural networkهای پیچیده. تنظیمات cudnn.deterministic از تغییرات تصادفی در GPU جلوگیری می‌کند تا نتایج در هر اجرا یکسان باشند.

</div>

In [ ]:
def smooth_signal(x, kernel=SMOOTH_KERNEL):
    if kernel is None or kernel <= 1:
        return x.astype(np.float32, copy=False)
    k = np.ones(kernel, dtype=np.float32) / kernel
    return np.convolve(x.astype(np.float32, copy=False), k, mode='same')

def downsample_avg(signal, target_len):
    if len(signal) == target_len:
        return signal.astype(np.float32, copy=False)
    parts = np.array_split(signal, target_len)
    return np.array([p.mean(dtype=np.float32) for p in parts], dtype=np.float32)

def clip_signal(signal, clip_percentile=99.5):
    clip_value = np.percentile(np.abs(signal), clip_percentile)
    return np.clip(signal, -clip_value, clip_value)

def frequency_augmentation(signal, scale_range=(0.85, 1.15)):
    freq = np.fft.fft(signal)
    scale = np.random.uniform(*scale_range)
    freq = freq * scale
    return np.fft.ifft(freq).real.astype(np.float32)

def mixup_data(x, y, alpha=0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size(0)
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

<div dir="rtl">

## Preprocessing Functions

این بخش functionهایی را برای preprocess سیگنال‌ها تعریف می‌کند: smooth_signal برای کاهش نویز با میانگین متحرک، downsample_avg برای کاهش طول سیگنال به target_len، clip_signal برای محدود کردن مقادیر extreme، frequency_augmentation برای تغییر مقیاس در domain فرکانس، و mixup_data برای ترکیب sampleها در train.

هر function یک سیگنال ورودی می‌گیرد و آن را transform می‌کند. مثلاً smooth_signal از convolution با یک uniform kernel برای averaging استفاده می‌کند، و mixup_data دو sample را با وزن تصادفی ترکیب می‌کند.

سیگنال به float32 تبدیل می‌شود، عملیات مورد نظر (مثل convolution، FFT، یا interpolation) اعمال می‌شود، و خروجی با type مناسب بازگشت داده می‌شود. برای mixup، دو نمونه با نسبت lam ترکیب می‌شوند.

این functionها سیگنال‌ها را برای input به مدل آماده می‌کنند. smoothing نویزهای کوچک را حذف می‌کند، downsampling طول سیگنال را استاندارد می‌کند، clipping از تأثیر outliers جلوگیری می‌کند، و augmentationها (frequency و mixup) تنوع data را افزایش می‌دهند تا model بهتر generalize کند و overfitting کاهش یابد.


</div>

In [ ]:
def get_all_files(base_dir):
    files = list(base_dir.rglob(f"*/{REVERSED_FOLDER}/*.csv"))
    return sorted([f for f in files if f.is_file()])

def find_degradation_start(signals):
    kurt = np.array([kurtosis(s) for s in signals])
    kurt = smooth_signal(kurt, kernel=5)
    normal_end = min(50, len(kurt) // 5)
    mean_normal = np.mean(kurt[:normal_end])
    std_normal = np.std(kurt[:normal_end])
    threshold = mean_normal + 3 * std_normal
    for i in range(normal_end, len(kurt)):
        if kurt[i] > threshold:
            return i
    return len(kurt) // 2

def preprocess_and_cache(files):
    print("Preprocessing & Caching (once)...")
    X_list, y_list = [], []
    max_rul = 0.0
    for f in tqdm(files):
        df = pd.read_csv(f, header=None, dtype=np.float32)
        sigs = df.iloc[:, :-1].values
        labels = df.iloc[:, -1].values.astype(np.float32)
        if labels.size == 0:
            continue
        deg_start = find_degradation_start(sigs)
        rul_at_start = labels[deg_start]
        adjusted_labels = labels.copy()
        adjusted_labels[:deg_start] = rul_at_start
        adjusted_labels = np.clip(adjusted_labels, 0, RUL_CLIP)
        max_rul = max(max_rul, float(adjusted_labels.max()))
        for sig, lab in zip(sigs, adjusted_labels):
            sig = clip_signal(sig)
            if SMOOTH_KERNEL > 1:
                sig = smooth_signal(sig, SMOOTH_KERNEL)
            sig_ds = downsample_avg(sig, TARGET_LEN)
            X_list.append(sig_ds)
            y_list.append(lab)
    X = np.asarray(X_list, dtype=np.float32)
    y = np.asarray(y_list, dtype=np.float32)
    np.save(CACHE_DIR / "X_raw.npy", X)
    np.save(CACHE_DIR / "y.npy", y)
    np.save(CACHE_DIR / "max_rul.npy", np.array([max_rul], dtype=np.float32))
    print(f"Cached raw: X={X.shape}, y={y.shape}, max_rul={max_rul:.2f}")

def load_cached_raw():
    X = np.load(CACHE_DIR / "X_raw.npy")
    y = np.load(CACHE_DIR / "y.npy")
    max_rul = float(np.load(CACHE_DIR / "max_rul.npy")[0])
    return X, y, max_rul

def split_indices(n_samples, test_ratio=TEST_RATIO, val_ratio=VAL_RATIO, seed=SEED):
    idx = np.arange(n_samples)
    rng = np.random.RandomState(seed)
    rng.shuffle(idx)
    n_test = int(test_ratio * n_samples)
    n_val = int(val_ratio * (n_samples - n_test))
    return idx[n_test + n_val:], idx[n_test:n_test + n_val], idx[:n_test]

def compute_train_norm_stats(X_train):
    mean = X_train.mean(axis=0, dtype=np.float64).astype(np.float32)
    std = X_train.std(axis=0, dtype=np.float64).astype(np.float32)
    std[std < 1e-6] = 1.0
    return mean, std

<div dir="rtl">

## Data Loading and Preprocessing Functions

این بخش شامل functionهایی برای یافتن فایل‌های CSV، تشخیص نقطه شروع degradation با kurtosis، preprocess و cache کردن data، load کردن dataهای cached، تقسیم indices برای train/validation/test، و محاسبه mean و std برای normalization.

get_all_files تمام فایل‌های CSV در directory مشخص را پیدا می‌کند. find_degradation_start از kurtosis برای شناسایی شروع تخریب استفاده می‌کند. preprocess_and_cache داده‌ها را می‌خواند، سیگنال‌ها را preprocess می‌کند (clip، smooth، downsample)، labels را تنظیم می‌کند، و در فایل‌های npy ذخیره می‌کند. load_cached_raw این فایل‌ها را می‌خواند. split_indices داده‌ها را به سه بخش تقسیم می‌کند، و compute_train_norm_stats آماری برای normalization محاسبه می‌کند.

فایل‌ها اسکن می‌شوند، data خوانده و transform می‌شود، labels بر اساس degradation point تنظیم و clipped می‌شوند، max_rul محاسبه می‌شود، و data ذخیره می‌شود. در load، فایل‌های npy خوانده می‌شوند. indices به‌صورت random با seed تقسیم می‌شوند، و mean/std محاسبه می‌شوند.

caching سرعت را افزایش می‌دهد، چون preprocess فقط یک بار انجام می‌شود. تشخیص degradation point با kurtosis labels را دقیق‌تر می‌کند، چون قبل از تخریب RUL ثابت فرض می‌شود. تقسیم data برای train، validation، و test ضروری است تا مدل ارزیابی شود. normalization مقیاس featureها را متعادل می‌کند تا training سریع‌تر و پایدارتر باشد.

- **سقف‌گذاری (Clipping) برچسب‌ها (RUL)**:
  - برچسب‌های RUL با استفاده از `np.clip` به بازه [۰, ۱۳۰] محدود می‌شوند (RUL_CLIP=۱۳۰).
  - **دلیل استفاده**: سقف‌گذاری RUL برای جلوگیری از تأثیر مقادیر بسیار بزرگ RUL (که ممکن است نویزی یا غیرقابل اعتماد باشند) بر عملکرد مدل است. این تکنیک به مدل کمک می‌کند تا روی مراحل نزدیک به شکست تمرکز کند، بیش‌برازش را کاهش دهد و دقت پیش‌بینی در مراحل بحرانی را افزایش دهد. این روش در پیش‌بینی RUL برای جلوگیری از overestimation و بهبود تعمیم‌پذیری مدل رایج است.
  - **نمونه مقالات استفاده‌کننده از سقف‌گذاری در دیتاست XJTU-SY**:
    - Ding et al. (۲۰۲۲) در مقاله "Prediction of remaining useful life of rolling bearing based on fractal dimension and convolutional neural network" (منتشر در Measurement and Control) از clipping RUL در دیتاست XJTU-SY برای بهبود دقت پیش‌بینی CNN استفاده کرده‌اند.
    - Lin (۲۰۲۱) در مقاله "A Novel Approach of Label Construction for Predicting Remaining Useful Life of Machinery" (منتشر در Shock and Vibration) از روش‌های clipping و تنظیم piecewise linear در XJTU-SY bearing data برای بهبود label construction و RUL prediction بهره برده‌اند.
    - Wang et al. (۲۰۲۲) در مقاله "RUL prediction of rolling bearings based on improved variational mode decomposition and LSTM" (منتشر در Advances in Mechanical Engineering) از capping RUL در XJTU-SY برای تمرکز روی مراحل تخریب و کاهش خطا در مدل LSTM استفاده کرده‌اند.

</div>

In [ ]:
class BearingDataset(Dataset):
    def __init__(self, X, y, max_rul, mean, std, train=False, augment=AUGMENT):
        self.X_raw = X
        self.y_raw = y
        self.y = (y / max_rul).astype(np.float32)
        self.max_rul = max_rul
        self.mean, self.std = mean, std
        self.augment = augment and train
        self.noise_scale = 0.015
        self.shift_max = 12
        self.scale_range = (0.85, 1.15)
        self.freq_scale_range = (0.8, 1.2)

    def __len__(self):
        return len(self.X_raw)

    def __getitem__(self, idx):
        x = self.X_raw[idx].copy()
        if self.augment:
            noise = np.random.normal(0, self.noise_scale * self.std, size=x.shape).astype(np.float32)
            x = x + noise
            shift = np.random.randint(-self.shift_max, self.shift_max)
            x = np.roll(x, shift)
            scale = np.random.uniform(*self.scale_range)
            x = x * scale
            if np.random.random() < 0.5:
                x = frequency_augmentation(x, self.freq_scale_range)
        x = ((x - self.mean) / self.std).astype(np.float32)
        weight = max(1.0, (self.max_rul - self.y_raw[idx]) / 50.0)
        return torch.from_numpy(x).unsqueeze(0), torch.tensor(self.y[idx], dtype=torch.float32), torch.tensor(weight, dtype=torch.float32)

<div dir="rtl">

## Dataset Class

کلاس BearingDataset یک PyTorch Dataset برای مدیریت data تعریف می‌کند. labels را normalize می‌کند (تقسیم بر max_rul)، و در حالت train augmentationهایی مثل اضافه کردن noise، shift، scaling، و frequency augmentation اعمال می‌کند.

در __init__، data و labels ذخیره و normalize می‌شوند. در __getitem__، یک sample انتخاب می‌شود، اگر augment فعال باشد، transformهای تصادفی اعمال می‌شوند، سپس data normalize شده و به tensor تبدیل می‌شود. weight برای هر sample محاسبه می‌شود.

نمونه انتخاب می‌شود، کپی از data خام گرفته می‌شود، augmentationها (noise، shift، scale، frequency) با احتمال مشخص اعمال می‌شوند، data با mean و std normalize می‌شود، و خروجی به صورت tensor با weight بازگشت داده می‌شود.

این کلاس امکان batch loading با DataLoader را فراهم می‌کند. augmentation تنوع data را افزایش می‌دهد تا model روی dataهای unseen بهتر عمل کند. normalization مقیاس featureها را یکسان می‌کند تا gradient descent پایدارتر باشد. weightها برای تمرکز بیشتر روی sampleهایی که RUL کمتری دارند (نزدیک به failure) استفاده می‌شوند تا خطاها در این نقاط حساس کاهش یابد.

</div>

In [ ]:
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=8):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class OptimizedModel(nn.Module):
    def __init__(self, input_len=TARGET_LEN):
        super().__init__()
        self.conv1_large = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=15, padding=7),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32, momentum=0.1),
            nn.Dropout1d(0.1),
            nn.MaxPool1d(3)
        )
        self.conv1_small = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32, momentum=0.1),
            nn.Dropout1d(0.1),
            nn.MaxPool1d(3)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=7, padding=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128, momentum=0.1),
            nn.Dropout1d(0.15),
            nn.MaxPool1d(3)
        )
        self.se_block = SEBlock(128, reduction=8)
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(192, momentum=0.1),
            nn.Dropout1d(0.2),
            nn.MaxPool1d(3)
        )
        self.conv3_extra = nn.Sequential(
            nn.Conv1d(192, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256, momentum=0.1),
            nn.Dropout1d(0.25),
            nn.MaxPool1d(3)
        )
        self.conv4 = nn.Sequential(
            nn.Conv1d(256, 320, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(320, momentum=0.1),
            nn.AdaptiveAvgPool1d(16)
        )

        with torch.no_grad():
            dummy = torch.zeros(1, 1, input_len)
            out1 = self.conv1_large(dummy)
            out2 = self.conv1_small(dummy)
            out = torch.cat([out1, out2], dim=1)
            out = self.conv2(out)
            out = self.se_block(out)
            out = self.conv3(out)
            out = self.conv3_extra(out)
            out = self.conv4(out)
            output_length = out.size(2)
            output_channels = out.size(1)

        total_downsample = input_len // output_length
        self.shortcut = nn.Sequential(
            nn.Conv1d(1, output_channels, kernel_size=1, stride=total_downsample),
            nn.BatchNorm1d(output_channels, momentum=0.1)
        )

        self.gru = nn.GRU(input_size=320, hidden_size=160, num_layers=2, batch_first=True, bidirectional=True, dropout=0.3)
        self.attn = nn.MultiheadAttention(embed_dim=320, num_heads=8, dropout=0.15, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(320, 320),
            nn.ReLU(inplace=True),
            nn.LayerNorm(320),
            nn.Dropout(0.4),
            nn.Linear(320, 192),
            nn.ReLU(inplace=True),
            nn.LayerNorm(192),
            nn.Dropout(0.3),
            nn.Linear(192, 96),
            nn.ReLU(inplace=True),
            nn.LayerNorm(96),
            nn.Dropout(0.2),
            nn.Linear(96, 1)
        )

    def forward(self, x):
        residual = self.shortcut(x)
        x1 = self.conv1_large(x)
        x2 = self.conv1_small(x)
        x = torch.cat([x1, x2], dim=1)
        x = self.conv2(x)
        x = self.se_block(x)
        x = self.conv3(x)
        x = self.conv3_extra(x)
        x = self.conv4(x)
        x = x + residual
        x = x.permute(0, 2, 1)
        x, _ = self.gru(x)
        x = self.attn(x, x, x)[0]
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x.squeeze(1)

<div dir="rtl">

## Model

کلاس SEBlock یک squeeze-and-excitation block برای channel attention پیاده‌سازی می‌کند. کلاس OptimizedModel یک neural network پیچیده با convolutional layers چندمقیاس، SE block، residual connection، GRU برای sequence modeling، multi-head attention، و یک classifier چندلایه تعریف می‌کند.

در __init__، لایه‌ها تعریف می‌شوند. یک dummy input برای محاسبه output size لایه‌های conv استفاده می‌شود تا shortcut layer هماهنگ شود. در forward، input از conv layers، SE block، GRU، attention و classifier عبور می‌کند و output تک‌بعدی تولید می‌شود.

input ابتدا از دو مسیر conv (large و small kernel) پردازش می‌شود، outputs ترکیب می‌شوند، سپس از conv layers بعدی، SE block، و residual connection عبور می‌کند. بعد از permute، GRU توالی را پردازش می‌کند، attention featureها را وزن‌دهی می‌کند، و classifier RUL را پیش‌بینی می‌کند.

این architecture برای استخراج featureهای local (با conv) و global (با GRU و attention) از سیگنال‌های temporal طراحی شده است. multi-scale convها الگوهای مختلف را capture می‌کنند، SE block مهم‌ترین channelها را highlight می‌کند، residual connection از vanishing gradient جلوگیری می‌کند، و GRU/attention وابستگی‌های زمانی را مدل می‌کنند. این ترکیب برای پیش‌بینی دقیق RUL بهینه شده است.

##  تعریف مدل

مدل یک شبکه کانولوشنی یک‌بعدی (1D CNN) بهینه‌شده با GRU و مکانیزم توجه شامل موارد زیر است:

### لایه‌های کانولوشنی
- **conv1_large**:
  - ۳۲ فیلتر، اندازه هسته ۱۵، پدینگ ۷
  - MaxPooling با اندازه ۳، Dropout ۰.۱
- **conv1_small**:
  - ۳۲ فیلتر، اندازه هسته ۵، پدینگ ۲
  - MaxPooling با اندازه ۳، Dropout ۰.۱
- **ترکیب**:
  - خروجی `conv1_large` و `conv1_small` با `torch.cat` ترکیب شده و به ۶۴ کانال می‌رسد.
- **conv2**:
  - ۱۲۸ فیلتر، هسته ۷، پدینگ ۳
  - MaxPooling با اندازه ۳، Dropout ۰.۱۵
- **conv3**:
  - ۱۹۲ فیلتر، هسته ۵، پدینگ ۲
  - MaxPooling با اندازه ۳، Dropout ۰.۲
- **conv3_extra**:
  - ۲۵۶ فیلتر، هسته ۵، پدینگ ۲
  - MaxPooling با اندازه ۳، Dropout ۰.۲۵
- **conv4**:
  - ۳۲۰ فیلتر، هسته ۳، پدینگ ۱
  - `AdaptiveAvgPool1d` به طول ۱۶

### بلوک Squeeze-and-Excitation (SE)
- پس از `conv2`:
  - کاهش ۸ کانالی برای تنظیم پویای کانال‌ها

### Shortcut
- مسیر مستقیم با کانولوشن ۱×۱ و BatchNorm برای ادغام با خروجی اصلی

### لایه GRU
- اندازه ورودی: ۳۲۰
- اندازه پنهان: ۱۶۰
- تعداد لایه‌ها: ۲
- دوجهته
- Dropout: ۰.۳

### مکانیزم توجه (Attention)
- اندازه ورودی: ۳۲۰
- تعداد سرهای توجه: ۸
- Dropout: ۰.۱۵

### لایه‌های Fully Connected
- لایه‌های خطی:
  - ۳۲۰ → ۳۲۰ → ۱۹۲ → ۹۶ → ۱
  - هر لایه با ReLU، LayerNorm و Dropout (نسبت‌های ۰.۵، ۰.۴، ۰.۳، ۰.۲)

### جریان داده
- خروجی‌های کانولوشنی با اتصال کوتاه جمع شده، سپس به GRU منتقل شده، با مکانیزم توجه پردازش شده و در نهایت به طبقه‌بندی‌کننده ارسال می‌شوند.

</div>

In [ ]:
def smape_loss(pred, y):
    return torch.mean(2 * torch.abs(pred - y) / (torch.abs(pred) + torch.abs(y) + 1e-8))

def huber_loss(pred, y, delta=1.0):
    err = pred - y
    abs_err = torch.abs(err)
    quadratic = torch.clamp(abs_err, max=delta)
    linear = abs_err - quadratic
    return torch.mean(0.5 * quadratic**2 + delta * linear)

def fit(model, train_loader, val_loader, max_epochs, base_lr, min_lr, warmup_epochs, weight_decay, patience, device, swa_start):
    if USE_HUBER_LOSS:
        criterion = huber_loss
    else:
        criterion = nn.MSELoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=max_epochs - warmup_epochs, eta_min=min_lr)
    swa_model = AveragedModel(model)
    swa_scheduler = SWALR(optimizer, swa_lr=base_lr * 0.1)

    model = model.to(device)
    best_val = float('inf')
    best_ep = -1
    history = {"train": [], "val": [], "val_smape": []}

    for epoch in range(1, max_epochs + 1):
        print(f"\n===== Epoch {epoch}/{max_epochs} =====")
        if epoch <= warmup_epochs:
            lr = base_lr * (epoch / warmup_epochs)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        model.train()
        train_loss, n = 0, 0
        optimizer.zero_grad()
        for i, (X, y, w) in enumerate(tqdm(train_loader, desc="Train", leave=False)):
            X, y, w = X.to(device), y.to(device), w.to(device)
            if np.random.random() < 0.4:
                X, y_a, y_b, lam = mixup_data(X, y, alpha=0.2)
                pred = model(X)
                if USE_HUBER_LOSS:
                    loss_a = huber_loss(pred, y_a)
                    loss_b = huber_loss(pred, y_b)
                    mse_loss = lam * loss_a + (1 - lam) * loss_b
                else:
                    mse_loss = lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)
                smape = lam * smape_loss(pred, y_a) + (1 - lam) * smape_loss(pred, y_b)
            else:
                pred = model(X)
                if USE_HUBER_LOSS:
                    mse_loss = huber_loss(pred, y)
                else:
                    mse_loss = criterion(pred, y)
                smape = smape_loss(pred, y)

            l1_loss = torch.mean(torch.abs(pred - y)) * 0.02
            loss = mse_loss + 0.03 * smape + l1_loss
            loss = loss / ACCUM_STEPS
            loss.backward()

            if (i + 1) % ACCUM_STEPS == 0 or (i + 1) == len(train_loader):
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
                optimizer.step()
                optimizer.zero_grad()

            train_loss += loss.item() * X.size(0) * ACCUM_STEPS
            n += X.size(0)

        train_loss /= n

        model.eval()
        val_loss, val_smape, n = 0, 0, 0
        with torch.no_grad():
            for X, y, _ in tqdm(val_loader, desc="Val", leave=False):
                X, y = X.to(device), y.to(device)
                pred = model(X)
                if USE_HUBER_LOSS:
                    loss = huber_loss(pred, y)
                else:
                    loss = criterion(pred, y)
                val_loss += loss.item() * X.size(0)
                val_smape += (2 * torch.abs(pred - y) / (torch.abs(pred) + torch.abs(y) + 1e-8)).mean().item() * X.size(0)
                n += X.size(0)
        val_loss /= n
        val_smape = 100 * val_smape / n

        history["train"].append(train_loss)
        history["val"].append(val_loss)
        history["val_smape"].append(val_smape)
        print(f"Train={train_loss:.4f} | Val={val_loss:.4f} | Val sMAPE={val_smape:.2f}%")

        if epoch > warmup_epochs:
            if epoch < swa_start:
                scheduler.step()
            else:
                swa_model.update_parameters(model)
                swa_scheduler.step()

        if val_loss < best_val - 1e-5:
            best_val = val_loss
            best_ep = epoch
            torch.save(model.state_dict(), CACHE_DIR / "best_model.pth")
            print("(best).")
        elif epoch - best_ep >= patience:
            print("Early stopping.")
            break

    torch.save(swa_model.state_dict(), CACHE_DIR / "swa_model.pth")
    return history, swa_model

<div dir="rtl">

## Loss and Training Functions

این بخش دو loss function تعریف می‌کند: sMAPE برای محاسبه خطای نسبی و Huber loss برای مقاومت در برابر outliers. تابع fit فرایند train مدل را مدیریت می‌کند با استفاده از AdamW optimizer، cosine annealing scheduler، stochastic weight averaging (SWA)، mixup augmentation، gradient accumulation، و early stopping.

sMAPE خطای نسبی را محاسبه می‌کند، Huber loss ترکیبی از MSE و MAE است. در fit، model به device منتقل می‌شود، optimizer و scheduler تنظیم می‌شوند، و در هر epoch، data از train_loader خوانده می‌شود، loss محاسبه می‌شود (با mixup در 40% موارد)، gradientها accumulate و clip می‌شوند، و validation loss و sMAPE محاسبه می‌شوند.

model به device می‌رود، loop روی epochs اجرا می‌شود. در warmup، learning rate به‌تدریج افزایش می‌یابد. در هر batch، mixup با احتمال اعمال می‌شود، loss ترکیبی (MSE/Huber + sMAPE + L1) محاسبه و backward می‌شود. gradientها پس از ACCUM_STEPS step می‌شوند. validation metrics محاسبه و history ذخیره می‌شود. اگر val_loss بهبود یابد، model ذخیره می‌شود، و SWA بعد از swa_start فعال می‌شود.

Huber loss نسبت به MSE در برابر outliers مقاوم‌تر است، و sMAPE خطای نسبی را برای ارزیابی بهتر فراهم می‌کند. fit از warmup برای stability اولیه، gradient accumulation برای effective batch size بزرگ‌تر، SWA برای generalization بهتر، و early stopping برای جلوگیری از overfitting استفاده می‌کند. ترکیب lossها دقت model را در نقاط حساس بهبود می‌دهد.

## ۳. یادگیری مدل

### بهینه‌سازی
- بهینه‌ساز: **AdamW**
  - نرخ یادگیری اولیه: `3e-4`
  - حداقل نرخ یادگیری: `1e-6`
  - گرم‌کردن برای ۱۰ Epoch اول
  - وزن‌دهی: `1e-4`
- تابع هزینه: **Huber Loss** با دلتا ۱.۰ (برای مقاومت در برابر نویز) همراه با:
  - sMAPE با وزن ۰.۰۳
  - L1 Loss با وزن ۰.۰۲
- محدود کردن گرادیان: حداکثر نورم ۰.۵
- انباشت گرادیان: با ACCUM_STEPS=۲

### داده‌افزایی در آموزش
- **Mixup**:
  - آلفا: ۰.۲
  - احتمال: ۴۰٪

### برنامه‌ریزی نرخ یادگیری
- **CosineAnnealingLR**:
  - T_max: ۲۹۰ (EPOCHS - WARMUP_EPOCHS)
  - حداقل نرخ: `1e-6`
- **Stochastic Weight Averaging (SWA)**: شروع از Epoch ۱۵۰ با نرخ یادگیری ۰.۱ برابر پایه

### (Early Stopping)
- توقف آموزش پس از ۱۰۰ Epoch بدون بهبود.

### حفظ بهترین مدل
- ذخیره در: `best_model.pth` (بر اساس کمترین خطای اعتبارسنجی) و `swa_model.pth` برای مدل SWA

</div>

In [ ]:
def tta_predict(model, x, n_aug=7):
    model.eval()
    preds = []
    with torch.no_grad():
        for _ in range(n_aug):
            x_aug = x.clone()
            if np.random.random() < 0.6:
                noise = torch.randn_like(x_aug) * 0.01
                x_aug += noise
            if np.random.random() < 0.5:
                shift = np.random.randint(-5, 5)
                x_aug = torch.roll(x_aug, shifts=shift, dims=-1)
            if np.random.random() < 0.5:
                scale = np.random.uniform(0.95, 1.05)
                x_aug = x_aug * scale
            pred = model(x_aug)
            preds.append(pred)
    return torch.stack(preds).mean(dim=0)

<div dir="rtl">

## Test-Time Augmentation (TTA)

این تابع test-time augmentation را پیاده‌سازی می‌کند، که در آن input چندین بار با transformهای تصادفی (noise، shift، scale) پیش‌بینی می‌شود و میانگین پیش‌بینی‌ها گرفته می‌شود.

model به حالت eval می‌رود، input کلون می‌شود، transformهای تصادفی با احتمال مشخص اعمال می‌شوند، پیش‌بینی برای هر نسخه گرفته می‌شود، و میانگین پیش‌بینی‌ها محاسبه می‌شود.

input چندین بار (n_aug=7) با تغییرات مثل اضافه کردن noise، shift زمانی، یا scaling پردازش می‌شود. هر نسخه از model پیش‌بینی می‌شود، و outputs در tensor stack شده و mean گرفته می‌شود.

TTA با averaging پیش‌بینی‌های چند نسخه augmented، variance پیش‌بینی را کاهش می‌دهد و accuracy را بدون نیاز به retraining افزایش می‌دهد. transformهای انتخاب‌شده مشابه augmentationهای train هستند تا model روی آنها robust باشد.

</div>

In [ ]:
def evaluate_test(model, test_loader, max_rul, use_swa=False, swa_model=None):
    if use_swa and swa_model is not None:
        model = swa_model
        model = model.to(device)  
        model.eval()
        print("Using SWA model for evaluation...")
    else:
        model.load_state_dict(torch.load(CACHE_DIR / "best_model.pth", map_location=device))
        model = model.to(device)
        print("Using best model for evaluation...")

    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test", leave=False):
            X, y, _ = batch
            X, y = X.to(device), y.to(device)
            if USE_TTA:
                pred = tta_predict(model, X, n_aug=7)
            else:
                pred = model(X)
            pred = pred.cpu().numpy() * max_rul
            true = y.cpu().numpy() * max_rul
            preds.extend(pred.tolist())
            trues.extend(true.tolist())

    preds, trues = np.array(preds), np.array(trues)
    mse = mean_squared_error(trues, preds)
    mae = mean_absolute_error(trues, preds)
    rmse = math.sqrt(mse)
    r2 = r2_score(trues, preds)
    smape = 100 * np.mean(2 * np.abs(preds - trues) / (np.abs(preds) + np.abs(trues) + 1e-8))
    mae_pct = 100 * mae / (trues.max() if trues.max() > 0 else 1.0)

    print(f"\n" + "="*60)
    print(f"FINAL TEST RESULTS (RUL_CLIP={RUL_CLIP}):")
    print(f"MSE={mse:.2f} | MAE={mae:.2f} | RMSE={rmse:.2f}")
    print(f"R²={r2:.4f} | sMAPE={smape:.2f}% | MAE%Max={mae_pct:.2f}%")
    print("="*60)

    plt.figure(figsize=(10, 5))
    plt.scatter(trues, preds, alpha=0.6, color='#1f77b4', edgecolors='w', linewidth=0.5, label="Predictions")
    plt.plot([trues.min(), trues.max()], [trues.min(), trues.max()], 'r--', lw=2, label="Ideal Fit")
    plt.xlabel("True RUL", fontsize=12)
    plt.ylabel("Predicted RUL", fontsize=12)
    plt.title("Predicted vs True RUL (Optimized Model — Article Ready)", fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    return preds, trues

<div dir="rtl">

## Test Evaluation

این تابع model را روی test data ارزیابی می‌کند، metrics مثل MSE، MAE، RMSE، R2، sMAPE و MAE%Max را محاسبه می‌کند، و یک scatter plot از پیش‌بینی‌ها در برابر مقادیر واقعی رسم می‌کند. اگر use_swa فعال باشد، از SWA model استفاده می‌کند.

model به device منتقل و به حالت eval می‌رود. برای هر batch در test_loader، پیش‌بینی‌ها (با یا بدون TTA) محاسبه می‌شوند، به مقیاس اصلی (با max_rul) برمی‌گردند، و در listهای preds و trues ذخیره می‌شوند. metrics با sklearn محاسبه و چاپ می‌شوند، و scatter plot رسم می‌شود.

model بارگذاری می‌شود (SWA یا best model)، test data در batchها پردازش می‌شود، پیش‌بینی‌ها جمع‌آوری و denormalize می‌شوند، metrics محاسبه و visualization انجام می‌شود.

این تابع performance واقعی model را روی unseen data ارزیابی می‌کند. metrics مختلف جنبه‌های مختلف خطا را پوشش می‌دهند (مثل MSE برای squared error، sMAPE برای relative error). scatter plot به تحلیل بصری خطاها کمک می‌کند. استفاده از SWA به دلیل generalization بهتر انتخاب شده است.

</div>

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history["train"], label="Train Loss", color='blue', lw=2)
    plt.plot(history["val"], label="Validation Loss", color='orange', lw=2)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training & Validation Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(history["val_smape"], label="Validation sMAPE%", color='green', lw=2)
    plt.xlabel("Epoch")
    plt.ylabel("sMAPE %")
    plt.title("Validation sMAPE Over Time")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

<div dir="rtl">

## Plot History

این تابع history مربوط به train loss، validation loss، و validation sMAPE را در دو subplot رسم می‌کند.

با matplotlib یک figure با دو subplot ایجاد می‌شود. train و val loss در subplot اول و val sMAPE در subplot دوم plot می‌شوند. labelها، grid، و legend اضافه می‌شوند.

figure initialize می‌شود، data از history استخراج و plot می‌شود، تنظیمات visualization (مثل title و label) اعمال می‌شوند، و plot نمایش داده می‌شود.

visualization از history کمک می‌کند تا روند train را monitor کنیم، convergence را بررسی کنیم، و علائم overfitting (مثل افزایش val loss) را تشخیص دهیم. sMAPE به‌عنوان metric نسبی برای تحلیل خطای relative مفید است.

</div>

In [ ]:
def run():
    if not (CACHE_DIR / "X_raw.npy").exists():
        files = get_all_files(BASE_DIR)
        if not files:
            raise RuntimeError("reversed not found")
        preprocess_and_cache(files)

    X_raw, y, max_rul = load_cached_raw()
    train_idx, val_idx, test_idx = split_indices(len(X_raw))
    mean, std = compute_train_norm_stats(X_raw[train_idx])

    train_ds = BearingDataset(X_raw[train_idx], y[train_idx], max_rul, mean, std, train=True)
    val_ds = BearingDataset(X_raw[val_idx], y[val_idx], max_rul, mean, std)
    test_ds = BearingDataset(X_raw[test_idx], y[test_idx], max_rul, mean, std)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=0)

    model = OptimizedModel(TARGET_LEN)
    history, swa_model = fit(model, train_loader, val_loader, EPOCHS, LR, MIN_LR, WARMUP_EPOCHS, WEIGHT_DECAY, PATIENCE, device, SWA_START)
    plot_history(history)

    print("\n=== Final Evaluation with SWA Model ===")
    evaluate_test(model, test_loader, max_rul, use_swa=True, swa_model=swa_model)

if __name__ == "__main__":
    run()

<div dir="rtl">

## Main Run

تابع run کل pipeline را اجرا می‌کند: بررسی و اجرای preprocess اگر cache وجود نداشته باشد، load data، تقسیم به train/val/test، ساخت datasetها و DataLoaderها، initialize و train مدل، رسم history، و evaluate نهایی با SWA model.

ابتدا بررسی می‌شود که cache وجود دارد یا خیر. اگر نه، preprocess_and_cache اجرا می‌شود. سپس data load و split می‌شود، datasetها و loaderها ساخته می‌شوند، model initialize و train می‌شود، history رسم و evaluate نهایی انجام می‌شود.

cache check می‌شود، data preprocess یا load می‌شود، indices تقسیم، datasetها و loaderها ساخته، model train شده، history visualize و test evaluate می‌شود.

این تابع کل workflow را به‌صورت یکپارچه مدیریت می‌کند تا اجرای کد آسان باشد. SWA در evaluate نهایی استفاده می‌شود چون معمولاً generalization بهتری نسبت به best model دارد. این ساختار modular امکان debug و تغییر را آسان می‌کند.
### نتایج تست
- **معیارها** (با مدل SWA و RUL_CLIP=۱۳۰):
  - MSE: ۱۱۸.۸۰
  - MAE: ۳.۵۳
  - RMSE: ۱۰.۹۰
  - R²: ۰.۹۰۸۲
  - sMAPE: ۵.۸۳٪
  - MAE%Max: ۲.۷۱٪
</div>